In [1]:
import sys
import pandas as pd
import itertools

path:str = '/Users/juanbeta/My Drive/Research/Energy/E-CVRP-TW/Code/'
#path: str = 'C:/Users/jm.betancourt/Documents/Research/Energy//E-CVRP-TW/Code/'

from E_CVRP_TW import  E_CVRP_TW, Feasibility
env = E_CVRP_TW(path)

sys.path.insert(0,path+'Experimentation/')
import plot_performance as plot

# Constructive

### Restrictive criterion

Testing various restriction configuration for the RCL

In [ ]:
criterions = ['distance', 'TimeWindow','Intra-Hybrid', 'Exo-Hybrid']
data_crit = dict()
for instance in env.instances:
    data_crit[instance] = dict()
    for criterion in criterions:
        data = plot.retrieve_const_performance(instance, path+f'Experimentation/Constructive/RCL criterion/{criterion}/')
        data_crit[instance][criterion] = dict() 
        data_crit[instance][criterion]['min_dist'] = data[0]
        data_crit[instance][criterion]['min_EV'] = data[1]

In [ ]:
print(' \t \t \tDist \t \t \t \t \tTW \t \t \t \t \tIntra-H \t \t \t \t \tExo-H')
print('Instance \tM \tEV \tFO \tgap \ts \t \tEV \tFO \tgap \ts \t \tEV \tFO \tgap \ts \t \tEV \tFO \tgap \ts')

objective = 'min_EV'

for instance in env.sizes['l'][:10]:
    env.load_data(instance)
    things = [instance, len(env.Costumers)]
    for criterion in criterions:
        things += [len(data_crit[instance][criterion][objective]['best individual']),
                    round(data_crit[instance][criterion][objective]['best distance'],2),
                    data_crit[instance][criterion][objective]['gap'],
                    round(data_crit[instance][criterion][objective]['time to find'],2),
                    ' ']
    print(*things,sep = '\t')

In [ ]:
instance = env.sizes['l'][4]
criterion = 'TimeWindow'
objective = 'min_EV'

print(f'########## Performance ##########')
print(f'incumbent: {round(data_crit[instance][criterion][objective]["best distance"],2)}')
print(f'gap: {round((data_crit[instance][criterion][objective]["best distance"] - env.bkFO[instance])/env.bkFO[instance]*100,2)}%')
print(f'time to find: {round(data_crit[instance][criterion][objective]["time to find"],2)}')

In [ ]:
instance = env.sizes['l'][6]
plot.plot_const_performance(data_crit[instance],instance, objective = 'min_EV')

In [ ]:
##### Export data to Excel
data = data_crit 

d = {'instance': [], 'M':[]}
for instance in env.instances:
    env.load_data(instance)
    d['instance'].append(instance)
    d['M'].append(len(env.Costumers))

for criterion in criterions:
    d[f'{criterion}/# EV'] = [len(data[instance][criterion]['min_EV']['best individual']) for instance in env.instances]
    d[f'{criterion}/FO'] = [round(data[instance][criterion]['min_EV']['best distance'],2) for instance in env.instances]
    # d[f'{criterion}/gap'] = [data[instance][criterion]['gap'] for instance in env.instances]
    d[f'{criterion}/t'] = [round(data[instance][criterion]['min_EV']['time to find'],2) for instance in env.instances]

df = pd.DataFrame(data = d)
df.to_excel('criterions.xlsx')

# Operators

## Darwinian phi rate

In [ ]:
operator = 'Darwinian phi rate'
# Configurations = {'Darwinian phi rate':{'penalization':['regular','cuadratic','cubic'],
#                                         'conservation proportion':[0.2, 0.4, 0.7],
#                                         'length restriction':[True, False],
#                                         }}

Configurations = {'Darwinian phi rate':{'penalization':['cuadratic','cubic'],
                                        'conservation proportion':[0.4, 0.7],
                                        'length restriction':[True, False],
                                        }}

keys = list(Configurations[operator].keys())
combinations = list(itertools.product(*[Configurations[operator][key] for key in keys]))
Grid = [{operator: {keys[i]: combination[i] for i in range(len(keys))}} for combination in combinations]

data_Darwin = dict()

for instance in env.instances:
    data_Darwin[instance] = dict()
    for ii in Grid:
        config = ii[operator]
        identifier = str()
        for a in Configurations[operator].keys():
            identifier += str(config[a])+str('_')
        identifier = identifier[:-1]
        data_Darwin[instance][identifier] = plot.retrieve_op_performance(instance, path+f'Experimentation/Operators/{operator}/results-{identifier}-{instance}')

In [ ]:
instance = env.sizes['l'][9]
plot.plot_op_performance(data_Darwin[instance],instance, objective = 'min_EV')

## evaluated insertion

In [14]:
operator = 'evaluated insertion'
Configurations = {'evaluated insertion':{'penalization':['regular','cuadratic','cubic'],
                                         'criterion':['random']}}
                                        #  'criterion':['Hybrid', 'phi rate', 'visited costumers']}}#, 'random']}}

keys = list(Configurations[operator].keys())
combinations = list(itertools.product(*[Configurations[operator][key] for key in keys]))
Grid = [{operator: {keys[i]: combination[i] for i in range(len(keys))}} for combination in combinations]

data_insert = dict()

for instance in env.instances:
    data_insert[instance] = dict()
    for ii in Grid:
        config = ii[operator]
        identifier = str()
        for a in Configurations[operator].keys():
            identifier += str(config[a])+str('_')
        identifier = identifier[:-1]
        data_insert[instance][identifier] = plot.retrieve_op_performance(path+f'Experimentation/Operators/{operator}/results-{identifier}-{instance}')

In [ ]:
instance = env.sizes['l'][0]
plot.plot_op_performance(data_insert[instance],instance, objective = 'min_EV')

In [2]:
Configurations = {'Darwinian phi rate':{'penalization':['cuadratic','cubic'],
                                        'conservation proportion':[0.4],
                                        'length restriction':[True, False],
                                        },
                  
                  'evaluated insertion':{'penalization':['regular','cuadratic','cubic'],
                                         'criterion':['random']},

                  'genetic parameters':{'population size':[1500,3000],
                                        'crossover rate':[0.3, 0.6],
                                        'mutation rate':[0.3, 0.6]}
                }

D_keys = list(Configurations['Darwinian phi rate'].keys()); D_combinations = list(itertools.product(*[Configurations['Darwinian phi rate'][key] for key in D_keys]))
e_keys = list(Configurations['evaluated insertion'].keys()); e_combinations = list(itertools.product(*[Configurations['evaluated insertion'][key] for key in e_keys]))
g_keys = list(Configurations['genetic parameters'].keys()); g_combinations = list(itertools.product(*[Configurations['genetic parameters'][key] for key in g_keys]))

Grid = [{'Darwinian phi rate': {D_keys[i]: D_combination[i] for i in range(len(D_keys))}, 
         'evaluated insertion': {e_keys[i]: e_combination[i] for i in range(len(e_keys))}, 
         'genetic parameters': {g_keys[i]: g_combination[i] for i in range(len(g_keys))}} for D_combination in D_combinations for e_combination in e_combinations for g_combination in g_combinations]

In [3]:
### Uploading experimentation!!!

test_instances = list()
for ii in [5,10,15]:
    for i in env.instances:
        env.load_data(i)
        if len(env.Costumers) == ii:
            test_instances.append(i)

experiments = range(20)


data_exp = dict()

for instance in test_instances:
    data_exp[instance] = dict()
    for num in experiments:
        data_exp[instance][num] = plot.retrieve_op_performance(path+f'Experimentation/Exp {num}/results-{instance}')

In [ ]:
##### Export data to Excel
list_idx = 2

d = {'instance': [], 'M':[]}#, 'const/#EV':[], 'const/FO':[], 'const/t':[]}
for instance in test_instances:
    env.load_data(instance)
    d['instance'].append(instance)
    d['M'].append(len(env.Costumers))

for num in experiments:
    d[f'{num}/# EV'] = [len(data_exp[instance][num][list_idx]['best individual']) for instance in test_instances]
    d[f'{num}/FO'] = [round(data_exp[instance][num][list_idx]['best distance'],2) for instance in test_instances]
    # d[f'{criterion}/gap'] = [data[instance][criterion]['gap'] for instance in env.instances]
    d[f'{num}/t'] = [round(data_exp[instance][num][list_idx]['time to find'],2) for instance in test_instances]


df = pd.DataFrame(data = d)
df.to_excel('oppp-results.xlsx')

In [ ]:
##### Export data to Excel
list_idx = 2

d = {'instance': [], 'M':[]}#, 'const/#EV':[], 'const/FO':[], 'const/t':[]}
for instance in env.instances:
    env.load_data(instance)
    d['instance'].append(instance)
    d['M'].append(len(env.Costumers))

for ii in Grid:
    config = ii[operator]
    identifier = str()
    for a in Configurations[operator].keys():
        identifier += str(config[a])+str('_')
    identifier = identifier[:-1]
    d[f'{identifier}/# EV'] = [len(data_insert[instance][identifier][list_idx]['best individual']) for instance in env.instances]
    d[f'{identifier}/FO'] = [round(data_insert[instance][identifier][list_idx]['best distance'],2) for instance in env.instances]
    # d[f'{criterion}/gap'] = [data[instance][criterion]['gap'] for instance in env.instances]
    d[f'{identifier}/t'] = [round(data_insert[instance][identifier][list_idx]['time to find'],2) for instance in env.instances]


df = pd.DataFrame(data = d)
df.to_excel('oppp-results.xlsx')